## <center> Non-linear equations - solving methods </center>

In [1]:
import numpy as np
import sympy as sym
from scipy import optimize as opt

### Symbolic solving

In [2]:
x, a, b = sym.symbols("x, a, b")
sol = sym.solve(a * sym.cos(x) - b * sym.sin(x), x)
sol

[-2*atan((b - sqrt(a**2 + b**2))/a), -2*atan((b + sqrt(a**2 + b**2))/a)]

In general nonlinear equations are typically not solvable analytically. For example, equations that contain both polynomial expressions and elementary functions, such as sinx = x, are often transcendental and do not have an algebraic solution. If we attempt to solve such an equation using SymPy, we obtain an error in the form of an exception:

    sym.solve(sympy.sin(x)-x, x) --> NotImplementedError: multiple generators [x, sin(x)]

No algorithms are implemented to solve equation -x + sin(x)


### Numeric solving - Optimization module
The SciPy optimize module provides multiple functions for numerical root finding. The optimize.bisect and optimize.newton functions implement variants of bisection and Newton methods. 

The optimize.bisect takes three arguments: first a Python function (e.g., a lambda function) that represents the mathematical function for the equation for which a root is to be calculated and the second and third arguments are the lower and upper values of the interval for which to perform the bisection method.


In [3]:
# Taking as example exp(x) -2
f_sol = opt.bisect(lambda x: np.exp(x) - 2, -2, 2)
print(f_sol)

0.6931471805601177


The SciPy optimize module provides additional functions for root finding. In particular, the optimize.brentq and optimize.brenth functions, which are variants of the bisection method, also work on an interval where the function changes sign.

In [4]:
# Newton method
x_root_guess = 0
f = lambda x: np.exp(x) - 2
f_newton = opt.newton(f, x_root_guess)
print(f_newton)

0.6931471805599592


In [5]:
fprime = lambda x: np.exp(x)  # the derivative is calculated

f_advanced_newton = opt.newton(f, x_root_guess, fprime=fprime)  # more accurate
print(f_advanced_newton)

0.6931471805599453


#### Systems of Nonlinear Equations
Taking as an example the following system of nonlinear equations:
$$ y - x^3-2x^2+1 = 0 $$
$$ y+x^2-1 = 0 $$
To solve this equation system using SciPy, we need to define a Python function for $f(x_1, x_2)$ and call, for example,
the optimize.fsolve using the function and an initial guess for the solution vector:

In [6]:
def f(var):
    x = var[0]
    y = var[1]
    return [y - x ** 3 - 2 * x ** 2 + 1, y + x ** 2 - 1]


guess = [1, 1]
sol, infodict, ier, mesg = opt.fsolve(f, guess, full_output=True)
sol

array([0.73205081, 0.46410162])

In [7]:
Jacobian = (infodict["fjac"])
eigenvalues, eigenvectors = np.linalg.eig(Jacobian)

print(eigenvalues) # Eigenvalues are calculated form the Jacobian matrix displayed from the fsolve function
print(eigenvectors)
 

[-0.95153319+0.30754606j -0.95153319-0.30754606j]
[[ 7.07106781e-01+0.j          7.07106781e-01-0.j        ]
 [-2.29934717e-17+0.70710678j -2.29934717e-17-0.70710678j]]


In [8]:
# To specify a Jacobian for optimize.fsolve to use, we need to define a function that evaluates the Jacobian for a
# given input vector. This requires that we first derive the Jacobian by hand or, for example, using SymPy

x, y = sym.symbols("x, y")
f_mat = sym.Matrix([y - x ** 3 - 2 * x ** 2 + 1, y + x ** 2 - 1])
jacob = f_mat.jacobian(sym.Matrix([x, y]))
jacob

Matrix([
[-3*x**2 - 4*x, 1],
[          2*x, 1]])

In [9]:
def jacobian(v):
    x = v[0]
    return [[-3 * x ** 2 - 4 * x, 1], [2 * x, 1]]


accu_sol = opt.fsolve(f, guess, fprime=jacobian)  # As it can be seen it's not necessary, results are the same
print(accu_sol)

[0.73205081 0.46410162]
